# Детектор углов Харриса и морфологические операции

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
from scipy import ndimage
from skimage import morphology, measure
import os

In [ ]:
# Загрузка тестового изображения
img_path = 'sample.jpg'

if os.path.exists(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print(f"✅ Изображение загружено: {img_path}")
else:
    print("📝 Создаём тестовое изображение с углами и геометрическими фигурами...")

    # Создаём изображение с чёткими углами для демонстрации
    img = np.zeros((400, 600, 3), dtype=np.uint8)

    # Прямоугольники (4 угла каждый)
    cv2.rectangle(img, (50, 50), (150, 120), (200, 200, 200), -1)
    cv2.rectangle(img, (200, 50), (350, 120), (150, 150, 150), 3)  # Только контур

    # L-образная фигура (углы разных типов)
    points = np.array([[400, 50], [500, 50], [500, 120], [450, 120], [450, 150], [400, 150]], np.int32)
    cv2.fillPoly(img, [points], (180, 180, 180))

    # Треугольники
    triangle1 = np.array([[100, 200], [150, 280], [50, 280]], np.int32)
    cv2.fillPoly(img, [triangle1], (160, 160, 160))

    # Пересекающиеся линии (создают углы в точках пересечения)
    cv2.line(img, (250, 180), (450, 280), (140, 140, 140), 3)
    cv2.line(img, (250, 280), (450, 180), (140, 140, 140), 3)

    # Ромб
    diamond = np.array([[350, 320], [400, 370], [350, 420], [300, 370]], np.int32)
    cv2.fillPoly(img, [diamond], (170, 170, 170))

    # Крест (множество углов)
    cv2.rectangle(img, (480, 320), (520, 420), (190, 190, 190), -1)  # Вертикальная часть
    cv2.rectangle(img, (460, 360), (540, 380), (190, 190, 190), -1)  # Горизонтальная часть

    # Добавляем немного шума
    noise = np.random.randint(-15, 15, img.shape, dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    print("✅ Синтетическое изображение создано")

# Преобразуем в grayscale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Отображаем исходное изображение
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title('Исходное изображение (RGB)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(img_gray, cmap='gray')
plt.title('Grayscale версия')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 1: Реализация детектора углов Харриса "с нуля"
# ===============================

def harris_corner_detector_manual(image, k=0.04, window_size=3, threshold=0.01):
    """
    Реализация детектора углов Харриса с нуля

    Args:
        image: входное изображение (grayscale)
        k: параметр Харриса (обычно 0.04-0.06)
        window_size: размер окна для вычисления матрицы моментов
        threshold: порог для классификации углов

    Returns:
        corners: карта откликов Харриса
        corner_points: координаты обнаруженных углов
    """

    print(f"🔄 Выполняем детекцию углов Харриса (k={k}, окно={window_size}x{window_size})...")

    # Шаг 1: Вычисляем частные производные (градиенты)
    print("   Шаг 1: Вычисление градиентов")

    # Операторы Собеля для вычисления градиентов
    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float32)

    # Вычисляем частные производные
    Ix = cv2.filter2D(image.astype(np.float32), -1, sobel_x)
    Iy = cv2.filter2D(image.astype(np.float32), -1, sobel_y)

    # Шаг 2: Вычисляем произведения градиентов
    print("   Шаг 2: Вычисление произведений градиентов")
    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    Ixy = Ix * Iy

    # Шаг 3: Применяем Гауссово окно для сглаживания
    print("   Шаг 3: Сглаживание с помощью гауссова окна")

    # Создаём гауссово ядро
    gaussian_kernel = cv2.getGaussianKernel(window_size, sigma=1)
    gaussian_kernel = gaussian_kernel @ gaussian_kernel.T

    # Применяем гауссово сглаживание к произведениям
    Sx2 = cv2.filter2D(Ix2, -1, gaussian_kernel)
    Sy2 = cv2.filter2D(Iy2, -1, gaussian_kernel)
    Sxy = cv2.filter2D(Ixy, -1, gaussian_kernel)

    # Шаг 4: Вычисляем функцию отклика Харриса
    print("   Шаг 4: Вычисление функции отклика")

    # Матрица моментов второго порядка M = [[Sx2, Sxy], [Sxy, Sy2]]
    # det(M) = Sx2 * Sy2 - Sxy^2
    # trace(M) = Sx2 + Sy2

    det_M = Sx2 * Sy2 - Sxy * Sxy
    trace_M = Sx2 + Sy2

    # Функция отклика Харриса: R = det(M) - k * trace(M)^2
    harris_response = det_M - k * (trace_M ** 2)

    # Шаг 5: Применяем пороговую фильтрацию
    print("   Шаг 5: Пороговая фильтрация")

    # Нормализуем отклик для применения порога
    harris_normalized = cv2.normalize(harris_response, None, 0, 1, cv2.NORM_MINMAX)

    # Находим углы (положительные значения выше порога)
    corners_mask = harris_normalized > threshold
    corner_points = np.argwhere(corners_mask)

    return harris_response, harris_normalized, corner_points, Ix, Iy

# Применяем наш детектор Харриса
harris_response, harris_norm, corner_points, grad_x, grad_y = harris_corner_detector_manual(
    img_gray, k=0.04, window_size=5, threshold=0.1
)

# Сравниваем с встроенной функцией OpenCV
harris_opencv = cv2.cornerHarris(img_gray, blockSize=5, ksize=3, k=0.04)
harris_opencv_norm = cv2.normalize(harris_opencv, None, 0, 1, cv2.NORM_MINMAX)

# Находим углы с помощью OpenCV
corner_points_cv = np.argwhere(harris_opencv_norm > 0.1)

print(f"📊 Результаты сравнения:")
print(f"   Наша реализация: {len(corner_points)} углов")
print(f"   OpenCV: {len(corner_points_cv)} углов")

# Визуализируем результаты
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Верхняя строка - промежуточные результаты
axes[0, 0].imshow(img_gray, cmap='gray')
axes[0, 0].set_title('Исходное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(grad_x, cmap='RdBu')
axes[0, 1].set_title('Градиент X (Ix)')
axes[0, 1].axis('off')

axes[0, 2].imshow(grad_y, cmap='RdBu')
axes[0, 2].set_title('Градиент Y (Iy)')
axes[0, 2].axis('off')

axes[0, 3].imshow(harris_response, cmap='jet')
axes[0, 3].set_title('Отклик Харриса (сырой)')
axes[0, 3].axis('off')

# Нижняя строка - финальные результаты
axes[1, 0].imshow(harris_norm, cmap='hot')
axes[1, 0].set_title('Наша реализация (нормализовано)')
axes[1, 0].axis('off')

axes[1, 1].imshow(harris_opencv_norm, cmap='hot')
axes[1, 1].set_title('OpenCV (нормализовано)')
axes[1, 1].axis('off')

# Отмечаем найденные углы на исходном изображении
img_with_corners = img_rgb.copy()
for point in corner_points:
    cv2.circle(img_with_corners, (point[1], point[0]), 3, (0, 255, 0), -1)

axes[1, 2].imshow(img_with_corners)
axes[1, 2].set_title(f'Найденные углы - наша реализация\n({len(corner_points)} углов)')
axes[1, 2].axis('off')

# OpenCV углы
img_with_corners_cv = img_rgb.copy()
for point in corner_points_cv:
    cv2.circle(img_with_corners_cv, (point[1], point[0]), 3, (255, 0, 0), -1)

axes[1, 3].imshow(img_with_corners_cv)
axes[1, 3].set_title(f'Найденные углы - OpenCV\n({len(corner_points_cv)} углов)')
axes[1, 3].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 2: Исследование параметров детектора Харриса
# ===============================

def harris_parameter_study(image):
    """Исследует влияние различных параметров на детектор Харриса"""

    # Различные значения параметра k
    k_values = [0.01, 0.04, 0.08, 0.15]

    # Различные размеры окна
    window_sizes = [3, 5, 7, 9]

    # Различные пороги
    thresholds = [0.05, 0.1, 0.2, 0.3]

    # Исследование влияния параметра k
    print("🔍 Исследование параметра k:")

    fig, axes = plt.subplots(1, 4, figsize=(16, 4))

    for i, k in enumerate(k_values):
        harris_resp = cv2.cornerHarris(image, blockSize=5, ksize=3, k=k)
        harris_norm = cv2.normalize(harris_resp, None, 0, 1, cv2.NORM_MINMAX)

        # Подсчитываем количество углов
        corners = np.sum(harris_norm > 0.1)

        axes[i].imshow(harris_norm, cmap='hot')
        axes[i].set_title(f'k = {k}\n{corners} углов')
        axes[i].axis('off')

        print(f"   k = {k}: {corners} углов обнаружено")

    plt.suptitle('Влияние параметра k на детекцию углов', fontsize=14)
    plt.tight_layout()
    plt.show()

    # Исследование влияния размера окна
    print("\n🔍 Исследование размера окна (blockSize):")

    fig, axes = plt.subplots(1, 4, figsize=(16, 4))

    for i, window_size in enumerate(window_sizes):
        harris_resp = cv2.cornerHarris(image, blockSize=window_size, ksize=3, k=0.04)
        harris_norm = cv2.normalize(harris_resp, None, 0, 1, cv2.NORM_MINMAX)

        corners = np.sum(harris_norm > 0.1)

        axes[i].imshow(harris_norm, cmap='hot')
        axes[i].set_title(f'Окно = {window_size}x{window_size}\n{corners} углов')
        axes[i].axis('off')

        print(f"   Окно {window_size}x{window_size}: {corners} углов обнаружено")

    plt.suptitle('Влияние размера окна на детекцию углов', fontsize=14)
    plt.tight_layout()
    plt.show()

# Проводим исследование параметров
harris_parameter_study(img_gray)

In [ ]:
# ===============================
# ЗАДАНИЕ 3: Основы морфологических операций
# ===============================

# Создаём тестовое бинарное изображение для демонстрации морфологических операций
def create_binary_test_image():
    """Создаёт тестовое бинарное изображение"""

    binary_img = np.zeros((300, 400), dtype=np.uint8)

    # Прямоугольники разных размеров
    binary_img[50:100, 50:150] = 255
    binary_img[120:180, 50:100] = 255

    # Круги
    cv2.circle(binary_img, (250, 75), 30, 255, -1)
    cv2.circle(binary_img, (320, 75), 15, 255, -1)

    # Линии (будут использованы для демонстрации морфологических операций)
    cv2.rectangle(binary_img, (50, 200), (350, 210), 255, -1)  # Горизонтальная линия
    cv2.rectangle(binary_img, (200, 150), (210, 250), 255, -1)  # Вертикальная линия

    # Добавляем шум (мелкие точки)
    noise_points = np.random.randint(0, 300, (20, 2))
    for point in noise_points:
        if point[0] < 300 and point[1] < 400:
            cv2.circle(binary_img, (point[1], point[0]), 1, 255, -1)

    # Создаём объект с "дырами"
    cv2.rectangle(binary_img, (280, 150), (350, 220), 255, -1)
    cv2.rectangle(binary_img, (295, 165), (335, 205), 0, -1)  # Дыра внутри

    return binary_img

# Создаём тестовое бинарное изображение
binary_test = create_binary_test_image()

# Также создаём бинарную версию нашего основного изображения
_, binary_main = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

print("✅ Тестовые бинарные изображения созданы")

# Отображаем тестовые изображения
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Исходное изображение (grayscale)')
axes[0].axis('off')

axes[1].imshow(binary_main, cmap='gray')
axes[1].set_title('Бинарная версия (Otsu)')
axes[1].axis('off')

axes[2].imshow(binary_test, cmap='gray')
axes[2].set_title('Тестовое бинарное изображение')
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 4: Реализация базовых морфологических операций
# ===============================

def manual_morphological_operations(image, kernel):
    """Реализация основных морфологических операций"""

    def erosion_manual(img, kern):
        """Ручная реализация эрозии"""
        result = np.zeros_like(img)
        kernel_center = kern.shape[0] // 2

        for i in range(kernel_center, img.shape[0] - kernel_center):
            for j in range(kernel_center, img.shape[1] - kernel_center):
                # Извлекаем область под ядром
                region = img[i-kernel_center:i+kernel_center+1,
                            j-kernel_center:j+kernel_center+1]

                # Эрозия: минимум под ядром (логическое И)
                if np.all(region[kern == 1] == 255):
                    result[i, j] = 255

        return result

    def dilation_manual(img, kern):
        """Ручная реализация дилатации"""
        result = np.zeros_like(img)
        kernel_center = kern.shape[0] // 2

        for i in range(kernel_center, img.shape[0] - kernel_center):
            for j in range(kernel_center, img.shape[1] - kernel_center):
                # Извлекаем область под ядром
                region = img[i-kernel_center:i+kernel_center+1,
                            j-kernel_center:j+kernel_center+1]

                # Дилатация: максимум под ядром (логическое ИЛИ)
                if np.any(region[kern == 1] == 255):
                    result[i, j] = 255

        return result

    # Выполняем операции
    eroded = erosion_manual(image, kernel)
    dilated = dilation_manual(image, kernel)

    # Составные операции
    opened = dilation_manual(eroded, kernel)  # Открытие = эрозия + дилатация
    closed = erosion_manual(dilated, kernel)  # Закрытие = дилатация + эрозия

    return eroded, dilated, opened, closed

# Создаём различные структурирующие элементы
kernels = {
    'Прямоугольник 3x3': np.ones((3, 3), np.uint8),
    'Прямоугольник 5x5': np.ones((5, 5), np.uint8),
    'Крест': np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]], dtype=np.uint8),
    'Круг': cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
}

print("🔧 Создано структурирующих элементов:", len(kernels))

# Применяем морфологические операции с разными ядрами
kernel_3x3 = np.ones((3, 3), np.uint8)

# Наши реализации
print("⚡ Применяем наши реализации...")
eroded_manual, dilated_manual, opened_manual, closed_manual = manual_morphological_operations(
    binary_test, kernel_3x3
)

# OpenCV реализации для сравнения
print("⚡ Применяем OpenCV...")
eroded_cv = cv2.erode(binary_test, kernel_3x3, iterations=1)
dilated_cv = cv2.dilate(binary_test, kernel_3x3, iterations=1)
opened_cv = cv2.morphologyEx(binary_test, cv2.MORPH_OPEN, kernel_3x3)
closed_cv = cv2.morphologyEx(binary_test, cv2.MORPH_CLOSE, kernel_3x3)

# Сравниваем результаты
operations = {
    'Исходное': binary_test,
    'Эрозия (наша)': eroded_manual,
    'Эрозия (OpenCV)': eroded_cv,
    'Дилатация (наша)': dilated_manual,
    'Дилатация (OpenCV)': dilated_cv,
    'Открытие (наше)': opened_manual,
    'Открытие (OpenCV)': opened_cv,
    'Закрытие (наше)': closed_manual,
    'Закрытие (OpenCV)': closed_cv
}

# Визуализируем результаты
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for i, (name, result) in enumerate(operations.items()):
    if i < len(axes):
        axes[i].imshow(result, cmap='gray')
        axes[i].set_title(name)
        axes[i].axis('off')

        # Подсчитываем белые пиксели
        white_pixels = np.sum(result == 255)
        total_pixels = result.shape[0] * result.shape[1]
        percentage = white_pixels / total_pixels * 100
        print(f"📊 {name}: {percentage:.1f}% белых пикселей")

plt.tight_layout()
plt.show()

# Проверяем точность наших реализаций
print(f"\n🔍 Проверка точности реализаций:")
print(f"Эрозия - совпадение: {np.array_equal(eroded_manual, eroded_cv)}")
print(f"Дилатация - совпадение: {np.array_equal(dilated_manual, dilated_cv)}")
print(f"Открытие - совпадение: {np.array_equal(opened_manual, opened_cv)}")
print(f"Закрытие - совпадение: {np.array_equal(closed_manual, closed_cv)}")

In [ ]:
# ===============================
# ЗАДАНИЕ 5: Продвинутые морфологические операции
# ===============================

def advanced_morphological_operations(image, kernel):
    """Продвинутые морфологические операции"""

    # Базовые операции
    eroded = cv2.erode(image, kernel, iterations=1)
    dilated = cv2.dilate(image, kernel, iterations=1)

    # Морфологический градиент = дилатация - эрозия
    gradient = cv2.subtract(dilated, eroded)

    # Top Hat (White Hat) = исходное - открытие
    opened = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    top_hat = cv2.subtract(image, opened)

    # Black Hat = закрытие - исходное
    closed = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    black_hat = cv2.subtract(closed, image)

    return gradient, top_hat, black_hat, opened, closed

# Применяем продвинутые операции
gradient, top_hat, black_hat, opened, closed = advanced_morphological_operations(
    binary_test, kernel_3x3
)

# Визуализируем продвинутые операции
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

operations_advanced = {
    'Исходное изображение': binary_test,
    'Морфологический градиент': gradient,
    'Top Hat (White Hat)': top_hat,
    'Открытие': opened,
    'Black Hat': black_hat,
    'Закрытие': closed
}

for i, (name, result) in enumerate(operations_advanced.items()):
    row, col = i // 3, i % 3
    axes[row, col].imshow(result, cmap='gray')
    axes[row, col].set_title(name)
    axes[row, col].axis('off')

    # Статистика
    white_pixels = np.sum(result == 255)
    print(f"📊 {name}: {white_pixels} белых пикселей")

plt.tight_layout()
plt.show()

In [ ]:
# ===============================
# ЗАДАНИЕ 6: Практическое применение - очистка изображений
# ===============================

def clean_binary_image(image, noise_size=2):
    """Очистка бинарного изображения от шума и дефектов"""

    # Создаём структурирующие элементы разных размеров
    small_kernel = np.ones((noise_size, noise_size), np.uint8)
    large_kernel = np.ones((noise_size*2, noise_size*2), np.uint8)

    # Этап 1: Удаление мелкого шума (открытие)
    step1_opened = cv2.morphologyEx(image, cv2.MORPH_OPEN, small_kernel)

    # Этап 2: Заполнение дыр (закрытие)
    step2_closed = cv2.morphologyEx(step1_opened, cv2.MORPH_CLOSE, large_kernel)

    # Этап 3: Сглаживание контуров (ещё одно открытие)
    step3_final = cv2.morphologyEx(step2_closed, cv2.MORPH_OPEN, small_kernel)

    return step1_opened, step2_closed, step3_final

# Создаём зашумленное изображение
noisy_image = binary_test.copy()

# Добавляем случайный шум
noise_mask = np.random.random(noisy_image.shape) < 0.05  # 5% пикселей
noisy_image[noise_mask] = 255 - noisy_image[noise_mask]  # Инвертируем случайные пиксели

# Добавляем "соль и перец"
salt = np.random.random(noisy_image.shape) < 0.02
pepper = np.random.random(noisy_image.shape) < 0.02
noisy_image[salt] = 255
noisy_image[pepper] = 0

print("🔧 Создано зашумленное изображение")

# Применяем очистку
cleaned_step1, cleaned_step2, cleaned_final = clean_binary_image(noisy_image)

# Визуализируем процесс очистки
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

cleaning_stages = {
    'Исходное (чистое)': binary_test,
    'Зашумленное': noisy_image,
    'После открытия': cleaned_step1,
    'После закрытия': cleaned_step2,
    'Финальный результат': cleaned_final,
    'Разность (шум)': cv2.absdiff(noisy_image, cleaned_final)
}

for i, (name, result) in enumerate(cleaning_stages.items()):
    row, col = i // 3, i % 3
    axes[row, col].imshow(result, cmap='gray')
    axes[row, col].set_title(name)
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

# Оценка качества очистки
original_pixels = np.sum(binary_test == 255)
noisy_pixels = np.sum(noisy_image == 255)
cleaned_pixels = np.sum(cleaned_final == 255)

print(f"\n📈 Оценка эффективности очистки:")
print(f"Исходное изображение: {original_pixels} белых пикселей")
print(f"Зашумленное: {noisy_pixels} белых пикселей")
print(f"После очистки: {cleaned_pixels} белых пикселей")
print(f"Точность восстановления: {100 - abs(original_pixels - cleaned_pixels)/original_pixels*100:.1f}%")

In [ ]:
# ===============================
# ЗАДАНИЕ 7: Анализ связанных компонент
# ===============================

def analyze_connected_components(image):
    """Анализ связанных компонент в бинарном изображении"""

    # Находим связанные компоненты
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=8)

    # Создаём цветное изображение для визуализации компонент
    colored_labels = np.zeros((image.shape[0], image.shape[1], 3), dtype=np.uint8)

    # Генерируем случайные цвета для каждой компоненты
    colors = np.random.randint(0, 255, (num_labels, 3))
    colors[0] = [0, 0, 0]  # Фон остаётся чёрным

    for label in range(num_labels):
        colored_labels[labels == label] = colors[label]

    return num_labels, labels, stats, centroids, colored_labels

# Анализируем компоненты в очищенном изображении
num_components, component_labels, component_stats, component_centroids, colored_components = analyze_connected_components(cleaned_final)

# Создаём изображение с информацией о компонентах
info_image = cv2.cvtColor(cleaned_final, cv2.COLOR_GRAY2RGB)

for i in range(1, num_components):  # Пропускаем фон (label = 0)
    # Рисуем центроид
    center = tuple(map(int, component_centroids[i]))
    cv2.circle(info_image, center, 3, (0, 255, 0), -1)

    # Добавляем номер компоненты
    cv2.putText(info_image, str(i),
                (center[0] + 5, center[1] - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

    # Рисуем ограничивающий прямоугольник
    x, y, w, h, area = component_stats[i]
    cv2.rectangle(info_image, (x, y), (x+w, y+h), (0, 0, 255), 1)

# Выводим статистику компонент
print(f"🔍 Найдено связанных компонент: {num_components - 1} (исключая фон)")

print(f"\n📊 Статистика компонент:")
for i in range(1, min(num_components, 11)):  # Показываем первые 10
    x, y, w, h, area = component_stats[i]
    center = component_centroids[i]
    print(f"   Компонента {i}: площадь={area}, центр=({center[0]:.1f}, {center[1]:.1f}), размер={w}x{h}")

# Визуализируем анализ компонент
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cleaned_final, cmap='gray')
axes[0].set_title('Очищенное изображение')
axes[0].axis('off')

axes[1].imshow(colored_components)
axes[1].set_title(f'Связанные компоненты\n({num_components-1} объектов)')
axes[1].axis('off')

axes[2].imshow(info_image)
axes[2].set_title('Анализ компонент\n(центроиды и ограничивающие прямоугольники)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# ДОМАШНЕЕ ЗАДАНИЕ

1. Реализуйте детектор углов FAST и сравните его с детектором Харриса по скорости и качеству детекции

2. Создайте морфологический фильтр для выделения линий определённой ориентации (горизонтальных или вертикальных)

3. Разработайте алгоритм автоматической настройки порога для детектора углов Харриса на основе анализа распределения откликов
